In [1]:
#segédfüggvények:


def txt_bemenet(txt):
    '''
    txt-t Sudoku bemenetnek megfelelő listává konvertál
    '''        
    fajl = open(txt, "rt").read().replace("\n"," ").split(" ")
    mezok = []
    i = 0
    for j in range(9):
        sor = []
        while fajl[i]!='':
            sor.append(int(fajl[i]))
            i +=1
        i += 1
        mezok.append(sor)
    return mezok

    
#a változók 0-tól 728-ig lesznek számozva, a mezők 1-től 81-ig, a rajtuk lévő értékek 1-től 9-ig
    
def valtozobol_mezo(n):
    '''
    változót konvertál mezővé és értékke
    '''
    x = math.floor(n/9)+1
    y = n%9+1
    return x, y

def mezobol_valtozo(x, y=1):
    '''
    mezőt és értéket konvertál változóvá
    '''
    return (x-1)*9+y-1


def ujmezo(beirt_mezok,l):
    '''
    az l listában lévő mezőkből megnézi, mit lehet egyértelműen kitalálni
    '''

    seged1 = pl.LpProblem(name="seged1")

    idx1 = [i for i in range(9**3)]
    M1 = pl.LpVariable.dicts(name="segéd1_mező",
                                indexs=idx1,
                                lowBound=0,
                                upBound=1,
                                cat=pl.LpBinary)

    rules = []
    for i in range(81):
        rules.append([j+9*i for j in range(9)])
    for i in range(9):
        for j in range(9):
            rules.append([9*k+81*j+i for k in range(9)])
    for i in range(9):
        for j in range(9):
            rules.append([81*k+9*j+i for k in range(9)])
    for i in range(9):
        for j in [0,3,6,27,30,33,54,57,60]:
            rules.append([9*k+9*j+i for k in [0,1,2,9,10,11,18,19,20]])

    for i in rules:
        seged1 += pl.lpSum(M1[j] for j in i) == 1

    for i in l:
        if i in beirt_mezok.keys():
            j = mezobol_valtozo(i, beirt_mezok[i])
            seged1 += M1[j] == 1

    if seged1.solve() != 1:
        return 0


    seged1_ertekek = [0 for i in range(729)]
    for i in idx1:
        seged1_ertekek[i] = pl.value(M1[i]) 
    ##eddig tart az első sudoku megoldása

    ##most megoldok még egyet, azzal a célfüggvénnyel, hogy minél több különbözzön

    seged2 = pl.LpProblem(name="seged2") # vagy pl.LpMinimize (ez a default)

    idx2 = [i for i in range(729)]
    M2 = pl.LpVariable.dicts(name="segéd2_mező",
                                indexs=idx2,
                                lowBound=0,
                                upBound=1,
                                cat=pl.LpBinary)


    rules = []
    for i in range(81):
        rules.append([j+9*i for j in range(9)])
    for i in range(9):
        for j in range(9):
            rules.append([9*k+81*j+i for k in range(9)])
    for i in range(9):
        for j in range(9):
            rules.append([81*k+9*j+i for k in range(9)])
    for i in range(9):
        for j in [0,3,6,27,30,33,54,57,60]:
            rules.append([9*k+9*j+i for k in [0,1,2,9,10,11,18,19,20]])

    for i in rules:
        seged2 += pl.lpSum(M2[j] for j in i) == 1

    for i in l:
        if i in beirt_mezok.keys():
            j = mezobol_valtozo(i, beirt_mezok[i])
            seged2 += M2[j] == 1   

    seged2 += pl.lpSum(M2[i]*seged1_ertekek[i] for i in idx2)


    seged2.solve()

    gyanus = []
    for i in range(729):
        x, y = valtozobol_mezo(i)
        if x not in beirt_mezok.keys() and seged1_ertekek[i] == 1 and pl.value(M2[i]) == seged1_ertekek[i]:
            gyanus.append(i)

    #most pedig megnézem, a gyanúsak között, melyikek vehetnek fel más értéket is    

    biztos = []
    for gyanu in gyanus:
        seged3 = pl.LpProblem(name="seged3") # vagy pl.LpMinimize (ez a default)

        idx3 = [i for i in range(729)]
        M3 = pl.LpVariable.dicts(name="segéd3_mező",
                                indexs=idx3,
                                lowBound=0,
                                upBound=1,
                                cat=pl.LpBinary)


        rules = []
        for i in range(81):
            rules.append([j+9*i for j in range(9)])
        for i in range(9):
            for j in range(9):
                rules.append([9*k+81*j+i for k in range(9)])
        for i in range(9):
            for j in range(9):
                rules.append([81*k+9*j+i for k in range(9)])
        for i in range(9):
            for j in [0,3,6,27,30,33,54,57,60]:
                rules.append([9*k+9*j+i for k in [0,1,2,9,10,11,18,19,20]])

        for i in rules:
            seged3 += pl.lpSum(M3[j] for j in i) == 1

        for i in l:
            if i in beirt_mezok.keys():
                j = mezobol_valtozo(i, beirt_mezok[i])
                seged3 += M3[j] == 1

        seged3 += M3[gyanu] == 0

        if seged3.solve() != 1:
            biztos.append(gyanu)

    return biztos


#ez még nincs kész, tulajdonképpen ennek a kitalálása a feladat lényege
def L_generalas(k):
    '''
    megad egy effektív L listát
    '''
    L = []
        
    db = math.floor(k/9)
    m = k%9
        
    sor1 = [1+i for i in range(9)]
    sor2 = [10+i for i in range(9)]
    sor3 = [19+i for i in range(9)]
    sor4 = [28+i for i in range(9)]
    sor5 = [37+i for i in range(9)]
    sor6 = [46+i for i in range(9)]
    sor7 = [55+i for i in range(9)]
    sor8 = [64+i for i in range(9)]
    sor9 = [73+i for i in range(9)]
    oszlop1 = [1+9*i for i in range(9)]
    oszlop2 = [2+9*i for i in range(9)]
    oszlop3 = [3+9*i for i in range(9)]
    oszlop4 = [4+9*i for i in range(9)]
    oszlop5 = [5+9*i for i in range(9)]
    oszlop6 = [6+9*i for i in range(9)]
    oszlop7 = [7+9*i for i in range(9)]
    oszlop8 = [8+9*i for i in range(9)]
    oszlop9 = [9+9*i for i in range(9)]
    harmas1 = [i for i in [1,2,3,10,11,12,19,20,21]]
    harmas2 = [3+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas3 = [6+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas4 = [27+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas5 = [30+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas6 = [33+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas7 = [54+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas8 = [57+i for i in [1,2,3,10,11,12,19,20,21]]
    harmas9 = [60+i for i in [1,2,3,10,11,12,19,20,21]]
        
    harmasok = [[harmas1,harmas2,harmas3,harmas4,harmas7],
                [harmas1,harmas2,harmas3,harmas5,harmas8],
                [harmas1,harmas2,harmas3,harmas6,harmas9],
                [harmas1,harmas4,harmas5,harmas6,harmas7],
                [harmas1,harmas4,harmas7,harmas8,harmas9],
                [harmas3,harmas4,harmas5,harmas6,harmas9],
                [harmas3,harmas6,harmas7,harmas8,harmas9],
                [harmas2,harmas5,harmas7,harmas8,harmas9],
                [harmas2,harmas4,harmas5,harmas6,harmas8]]
        
        
    for i in harmasok:
        l = []
        for j in i:
            for k in j:
                l.append(k)
        L.append(l)

    return L

In [5]:
import math
import pulp as pl
from random import randrange

class Sudoku:
    '''
    ez a Sudokuk osztálya
    '''
    
    def __init__(self,mezok=[[0 for i in range(9)] for i in range(9)]):
        
        '''
        minden Sudokut egy 9x9-es ,,listák egy listában,, szimbolizál
        '''
        
        self.mezok = mezok
        
        beirt_mezok = {}
        for i in range(9):
            for j in range(9):
                if mezok[i][j] != 0:
                    beirt_mezok[9*i+j+1] = mezok[i][j]
                    
        self.beirt_mezok = beirt_mezok
        
                
    def __str__(self):
        
        '''
        a gyakorlatról lopva
        '''
        
        allapot = " +" + 9 * "---+" + "\n"
        for i in range(9):
            sor = "|"
            for j in range(9):
                sor += f" {self.mezok[i][j]} |"
            allapot += f" {sor} \n +" + 9 * "---+" + "\n"
        
        return str(allapot)
    
    
    def uj_ertekek(self,mezok,ertekek):
        for i in range(len(mezok)):
            sor = math.ceil(mezok[i]/9)-1
            oszlop = mezok[i]%9-1
            self.mezok[sor][oszlop] = ertekek[i]
            self.beirt_mezok[mezok[i]] = ertekek[i]
        return self
       
    
    def megold_K(self,k):
        '''
        megoldja a Sudokut úgy, hogy mindig k mezőre emlékszik csak
        '''
        L = L_generalas(k)
        beirt_mezok = self.beirt_mezok.copy()
        
        verbose = True
        while verbose:
            verbose = False
            for l in L:
                uj_mezok = ujmezo(beirt_mezok,l)
                if uj_mezok == 0:
                    return "Nem lehet megoldani"
                for mezo in uj_mezok:
                    x, y = valtozobol_mezo(mezo)
                    beirt_mezok[x] = y
                if uj_mezok != []:
                    verbose = True

        if len(beirt_mezok) != 81:
            return "Nem tudja megoldani"
        
        self.beirt_mezok = beirt_mezok
        Sudoku.uj_ertekek(self,list(beirt_mezok.keys()),list(beirt_mezok.values()))
        
        return self

In [3]:
pelda1 = Sudoku(txt_bemenet("s01a.txt"))
print(pelda1)
Sudoku.megold_K(pelda1,8)
print(pelda1)

In [9]:
pelda2_lista = [
                    [0,3,0,2,7,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0],
                    [8,0,0,0,0,0,0,0,0],
                    [5,1,0,0,0,0,0,8,4],
                    [4,0,0,5,9,0,0,7,0],
                    [2,9,0,0,0,0,0,1,0],
                    [0,0,0,0,0,0,1,0,5],
                    [0,0,6,3,0,8,0,0,7],
                    [0,0,0,0,0,0,3,0,0]
                ]

pelda2 = Sudoku(pelda2_lista)
print(pelda2)
Sudoku.megold_K(pelda2,8)

 +---+---+---+---+---+---+---+---+---+
 | 0 | 3 | 0 | 2 | 7 | 0 | 0 | 0 | 0 | 
 +---+---+---+---+---+---+---+---+---+
 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 
 +---+---+---+---+---+---+---+---+---+
 | 8 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 
 +---+---+---+---+---+---+---+---+---+
 | 5 | 1 | 0 | 0 | 0 | 0 | 0 | 8 | 4 | 
 +---+---+---+---+---+---+---+---+---+
 | 4 | 0 | 0 | 5 | 9 | 0 | 0 | 7 | 0 | 
 +---+---+---+---+---+---+---+---+---+
 | 2 | 9 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 
 +---+---+---+---+---+---+---+---+---+
 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 5 | 
 +---+---+---+---+---+---+---+---+---+
 | 0 | 0 | 6 | 3 | 0 | 8 | 0 | 0 | 7 | 
 +---+---+---+---+---+---+---+---+---+
 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 
 +---+---+---+---+---+---+---+---+---+



'Nem tudja megoldani'